## Analysis of Gravitational Wave data from LIGO

Today we are going to follow the analysis of the LIGO data showing the first detection of gravitational waves.  You can find more information here:  https://losc.ligo.org/events/GW150914/ .  This notebook is adapted from one produced by the LIGO Open Science Center, https://losc.ligo.org/about/ , which has a variety of resources for learning about analyzing LIGO data. 

Our first step is to download the two datafiles at these links:

https://losc.ligo.org/s/events/GW150914/H-H1_LOSC_4_V1-1126259446-32.hdf5

https://losc.ligo.org/s/events/GW150914/L-L1_LOSC_4_V1-1126259446-32.hdf5

Save those files in the same directory as this notebook. 

The names of the files tell us about the data:

* "H-H1" means that the data come from the LIGO Hanford Observatory site and the LIGO "H1" detector;

* the "4" means the strain time-series data are (down-)sampled from 16384 Hz to 4096 Hz.  There are also corresponding versions that haven't been binned, which have a "32" in place of the "4" there. 

* the "V1" means version 1 of this data release;

* "1126257414-4096" means the data starts at GPS time 1126257414 (Mon Sep 14 09:16:37 GMT 2015), duration 4096 seconds; NOTE: GPS time is number of seconds since Jan 6, 1980 GMT. See http://www.oc.nps.edu/oc2902w/gps/timsys.html or https://losc.ligo.org/gps/

* the filetype "hdf5" means the data are in hdf5 format: https://www.hdfgroup.org/HDF5/

Helpfully, the LIGO team has also released a set of python functions to read the LIGO data.  However, they were written for Python 2 only, and some of you are running Python 3.  If you have Python 2 (look in the upper-right-hand corner of this notebook) then you can grab the LIGO original version: 

https://losc.ligo.org/s/sample_code/readligo.py

If you're running Python 3, this version should work:

https://www.dropbox.com/s/ba4nxbxspobn7jy/readligo.py?dl=0

Either way, save the file as 'readligo.py'.  You'll then be able to load this set of functions in just the same way as you have in the past for `numpy` or `scipy`, using `import`.

Our next step is to read in the data using a function in `readligo` called `loaddata`, which returns the strain data, the time data, and a dictionary file of data quality channels.  Dictionaries are a type of python data, where instead of integer indices like arrays, the data in a dictionary can be accessed by key, which can be strings, numbers, or tuples.


#### Exercise 1

Import the `readligo` functions and use `loaddata` to load the data from H1 and L1 downsampled to 4096 Hz (which are the two data files listed above).  Note that you'll need to look at the code in `readligo.py` to see what input parameters that function takes, and what it returns (so that you can "catch" the return values appropriately in different variables). Remember that a function can return more than one thing, so you want to have an appropriate number of variables on the left-hand side of the function call. 

Plot the data in a window $\pm $5 s around the event.  The event we're interested occurred at $t=$1126259462.422, so set the x limits of your plot to be centered around this event. You may want to define a variable with the time of the event, since we'll be repeatedly zooming in on that region of the data.  An alternate way to do it is to subtract the event time from both of your time arrays, so that all of your times are now relative to the time of the event.  (You'll still need to set your plot limits, but if you adjust the times, it's much easier to specify what the limits need to be, e.g. `[-5, 5]` in this case.)

Plot both L1 and H1 together, in different colors, and label your axes appropriately. 

Despite there being a lot of variation here, you can't see the signal in these data - this is all noise.  To see it, we'll need to do some transformations on the data, but first let's see what the data look like in the frequency domain.  As we learned a few weeks ago, one way to do this is to Fourier transform the data. We could do that and plot the power spectrum, but there's a built-in function in `matplotlib` that will calculate the power spectrum (also known as the "power spectral density", or PSD) in a way that reduces its noise somewhat.  It does this by essentially calculating the power spectrum for different chunks of the data, and averaging them.  This smooths out random noise that varies from segment to segment of the data, and reinforces frequencies that are present in all parts of the data. 

#### Exercise 2

Use the function `matplotlib.mlab.psd()` to generate both the frequency array and the power spectral density array so that you can plot them.  Use a value of 4096 for both `Fs` (the sampling frequency of the data) and `NFFT` (the size of the chunks of data to separately Fourier transform and average).  Thus, we're breaking the signal up into 1-second chunks and averaging the power spectra of those chunks. 

By doing this averaging of the FTs of different chunks of the data, we are emphasizing those frequency components that repeat over and over, i.e. that are present in all parts of the data.  These tend to be noise sources rather than real astrophysical signals, so we'll use that fact later to help clean up the data.  For now, let's just calculate and plot it to get a sense of what we're dealing with.

Plot the square root of the power spectral density of both H1 and L1 between frequencies of 10 Hz and 2048 Hz.  Below 10 Hz, LIGO is not properly calibrated.  Above 2048 Hz, the LIGO data don't contain meaningful information.  This is because of the sampling rate of 4096 Hz.  For a given sampling rate, there is a frequency called the *Nyquist frequency*, which is half the sampling rate, so in this case it is 2048 Hz.  Signals varying faster than the Nyquist frequency don't leave a meaningful imprint in the data stream, since there are fewer than two samples per oscillation.  One way to think about this is to imagine sampling a cosine function.  If you only have one sample over a full oscillation period of the cosine, you can't learn anything about it.  Even two seems a little thin, but at least then you have some information that there is a signal that is varying on that timescale.  

So, to reiterate, plot the square root of the PSD of both strain signals as a function of frequency, limiting the plot to frequencies from 10 Hz to 2048 Hz. 

The sharp spectral lines you see are from the LIGO instruments. For instance, the peak at about 500 Hz and its harmonics (integer multiples) are the mirror suspension resonances.  You can also see specific sources of noise, e.g. at 60 Hz there is a strong peak - can you guess why? 

We can see that the data are very strongly "colored" - noise fluctuations are much larger at low and high frequencies and near spectral lines, reaching a roughly flat ("white") minimum in the band around 80 to 300 Hz.

To get rid of much of this noise, we can "whiten" the data.  We take advantage of the fact we discussed above, that the PSD we calculated contains only the repeating noise sources.  In contrast, the regular Fourier transform of all of the data contains information about the noise sources, but also about any other signals (including the transient ones we expect from gravitational wave sources).  So by dividing the FT by the averaged PSD (the noise spectrum) in the Fourier domain, we can clean out much of the noise, to better see the weak signals in the most sensitive band.

Whitening is often one of the first steps in astrophysical data analysis (searches, parameter estimation). Whitening requires no prior knowledge of spectral lines, etc; only the data are needed.  After whitening, the resulting time series is no longer in units of strain; it is now in units of "sigmas" away from the mean.

#### Exercise 3

Below we have given a function that does this whitening. 
For the function's arguments, strain is the original data from the file, and the power spectral density and its frequencies are what you found earlier.  Call the whitening function on both H1 and L1.  Plot the results, now zooming in on just $\pm$0.1 second around the time of the event. 

In [4]:
def whiten(strain, psd, psd_freqs, dt=1./4096):
    '''Given a vector of strain data, return a whitened version. Inputs are 
    strain, the vector of strain values; psd, the power spectral density (psd)
    and frequencies (psd_freqs) calculated from matplotlib.mlab.psd; and 
    optionally, the time spacing of the data (defaults to 1/4096).  
    Returns a whitened vector of strain data. '''
    
    from scipy.interpolate import interp1d
    
    # Calculate the freqs for the FT of the strain data:
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)

    # Calculate the interpolating function that we'll use to 
    # resample the PSD onto the same frequency grid as the PSD:
    interp_psd = interp1d(psd_freqs, psd)

    # Whitening: transform to freq domain, divide by asd, then transform back, 
    # taking care to get normalization right.

    strain_ft = np.fft.rfft(strain)
    white_hf = strain_ft / (np.sqrt(interp_psd(freqs) /dt/2.))
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

As you'll see above, it's not yet clear that we've made things better.  There is still a lot of high-frequency noise, which we need to clean up. 

#### Exercise 4

Run a bandpass filter on the data, which selects the range of frequencies we're interested in.  The `scipy.signal` library contains a function `butter()` that will generate a Butterworth filter - a filter that has a "flat" passband, and rejects signals outside that band.  It also has another function `filtfilt()` that allows you to apply the filter to the data.  Below is a function that wraps these two things together. Apply a bandpass filter between 20 and 300 Hz to the whitened versions of both data sets, and plot the results.

In [6]:
def bandpass(signal, freq_low, freq_high, sampling_rate=4096):
    '''Given an input signal, create and apply a bandpass filter to 
    pass only frequencies between freq_low and freq_high.  The signal 
    is assumed to be sampled at sampling_rate (default 4096 Hz). 
    Return a vector of the signal with the filter applied.'''
    
    import scipy.signal
    
    filter_num, filter_denom = scipy.signal.butter(4, [freq_low*2./sampling_rate, \
                                                       freq_high*2./sampling_rate], \
                                                       btype='band')
    whitened_signal = scipy.signal.filtfilt(filter_num, filter_denom, signal)
    return whitened_signal


#### Exercise 5

You can see that signals don't quite line up.  Because of the direction from which the signal arrived, it hit the L1 detectors 7 milliseconds earlier.  Also, because of the orientations of the arms of the two detectors, one of the signals is inverted relative to the other.  To fix both of these offsets, shift the L1 data by 7 milliseconds and invert them, and then plot both signals. You should then clearly see the gravitational wave signal registered by both detectors. 

#### Exercise 6

Now that we have the signal, we can compare it to a model of the event.  Calculations of these gravitational wave signals are generally done with computer calculations, a field called *numerical relativity*.  While calculating these models is well beyond the scope of this exercise, we can overplot the signal that the LIGO scientists calculated.  

Use the `numpy` function `genfromtxt` that we used last week to load the model, found here:

https://losc.ligo.org/s/events/GW150914/GW150914_4_NR_waveform.txt

After you have loaded it, whiten it and bandpass-filter it as you did for the strain data.  However, there is one subtlety here - in order for the model to have the same normalization as the data, we need to whiten it in the same way.  That means that in the whitening for the model strain data, you need to use the PSD and frequencies *from the data*.  That ensures that the model has been processed in the same way, and is then directly comparable to the data. 

This kind of processing of models is fairly common in physics and astronomy.  We have perfect models from theory, and imperfect observations that bear the imprint of the physical limitations of our telescopes, cameras, microscopes, detectors, etc.  So we need to "observe" the models in the same way as we have observed the real world, so that the instrumental imprint on both data and model is similar. 

Make a plot of the original model, before whitening and filtering. 

Then, make a second plot in which you overplot the whitened and bandpass-filtered model with the data, after correcting for the fact that the model is centered around time 0 (possibly with a very small shift with respect to the event time in the data). 

#### Exercise 7

The gravitational wave signal has a time-varying amplitude, at frequencies that are similar to those of sounds that our ears can hear.  So, while the gravitational waves didn't make a sound when they arrived at Earth, expressing that signal as a sound is another way for us to "visualize" the data (though that's not really the right verb here).  

To play the signal as a sound, take the processed strain data array and create a subset of it that contains a $\pm$ 2 second interval around the event.  If necessary, look back at our previous exercises to remember how to make a boolean mask that lets you create a subset of a `numpy` array that meets certain criteria.  

Take this smaller chunk of data and play it as a sound.  (You may need to look back at our previous exercise on Fourier transforms to remember how to play a sound file within the notebook.)  The sound won't be dramatic - it's a faint "thump" at the moment of the event. 

#### Exercise 8

Finally, let's make the signal a little easier to hear.  We'll be using a technique that astronomers use when they make images that map certain wavelengths of light into different colors so our eyes can see them better.  This is sometimes used to take invisible wavelengths like infrared and map them into visible colors (like in [this infrared image of the Milky Way](http://apod.nasa.gov/apod/ap150308.html)), or to take colors that are very similar to each other and make them more distinct (as in [this image of the Fishhead Nebula](http://apod.nasa.gov/apod/ap141224.html), also shown below, where emission from lines of nitrogen, hydrogen, and sulfur [the latter two of which would both appear red to us]) are mapped to different colors so we can better see the contrast, indicating different physical conditions. 

<img src="http://apod.nasa.gov/apod/image/1412/ic1795_snyder_2500.jpg" width="80%">

In our case, we will take the low-frequency signal you heard above, and shift it up in frequency so that we can hear it better.  To do this, we can Fourier transform the data, then shift all of the Fourier components a certain number of bins in the array, which moves them in frequency by a fixed amount.  When we inverse transform the data, sounds that were lower frequency will have been shifted to higher frequencies.  Shift the sound up by 400 Hz, and listen to it.  You may want to make use of `numpy.roll`, which is a generic function for shifting values within an array by a set number of steps.  (It also handles the elements that are shifted off the end of the array, wrapping them back around to the beginning.)